In [1]:
!pip install pandas_profiling  
from pandas_profiling import ProfileReport
import pandas as pd

  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27090 sha256=ad359acf190abad9756a9fe139e99df98e1b512984a9330bc776ae456440a3e8
  Stored in directory: c:\users\christine\appdata\local\pip\cache\wheels\23\14\6e\4be5bfeeb027f4939a01764b48edd5996acf574b0913fe5243
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295204 sha256=a633f2404d1b493ad4f90aa65e7fc572941233d029042f3d306cdce9321d6662
  Stored in directory: c:\users\christine\appdata\local\pip\cache\wheels\48\a1\7f\096c1269d6bf78d4768180602579b35a1e8cb1250bb4b40c74
Successfully built htmlmin imagehash
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.47.0
    Uninstalling tqdm-4.47.0:
      Successfully uninstalled tqdm-4.47.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.2
    Uninstalling typing-extensions-3.7.4.2:
      Successfully uninstalled typing-extensions-3.7.4.2
  Attempting uninstall: joblib
   

ERROR: phik 0.12.2 has requirement scipy>=1.5.2, but you'll have scipy 1.5.0 which is incompatible.


In [34]:
#data profiling
df = pd.read_csv('covid_restaurant_full.csv')
df_loc = pd.read_csv('covid_restaurant_full_loc.csv')
profile = df.profile_report(title='Full Data Profiling')
#export as html file
profile.to_file(output_file="Full Data Profiling.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
#Based on results from the report, remove duplicates
print(df.shape)
df2 = df.drop_duplicates()
df_loc_2 = df_loc.drop_duplicates()
print(df2.shape)
#Duplicates won't drop in this version of dataset since it includes up to fips zip and not just state and area
print(df_loc_2.shape)

(34117, 61)
(31715, 61)
(31715, 65)


In [26]:
#Based on results from report, examine some of the correlated variables more deeply and decide to drop or not
print(df2['total_restaurants_zip'].corr(df2['Total_Reviews_Zip']))
print(df2['Rural_urban_continuum_code_2013'].corr(df2['Urban_influence_code_2013'])) #.94 correlation, should remove

print(df2['Unemployment_rate_2020'].corr(df2['Median_Household_Income_2019']))
print(df2['Unemployment_rate_2020'].corr(df2['Med_HH_Income_Percent_of_State_Total_2019']))
print(df2['Median_Household_Income_2019'].corr(df2['Med_HH_Income_Percent_of_State_Total_2019'])) #.93 correlation, should remove

print(df2['Unemployment_rate_2020'].corr(df2['covid_moderate']))
print(df2['Unemployment_rate_2020'].corr(df2['covid_substantial']))
print(df2['Median_Household_Income_2019'].corr(df2['covid_substantial']))
print(df2['Median_Household_Income_2019'].corr(df2['covid_moderate']))

#From the report, many of the economic location/variables seem correlated with one another. Will run a VIF in a later step to 
#help better determine which ones to remove, for now keep them in the data

0.5236268683293631
0.8737431864434867
0.9436643320423324
-0.5332274394799003
-0.5706826463616347
0.935305426006936
0.6264971563856977
-0.5812055390563952
0.4987345927935435
-0.4388564085488304


In [36]:
#Based on results from report, examine some of the highly correlated category variables
print(df2['sushi'].corr(df2['japanese']))
print(df2['hotdogs'].corr(df2['burgers']))
print(df2['italian'].corr(df2['pizza']))
print(df2['bars'].corr(df2['sportsbars']))
print(df2['bars'].corr(df2['juicebars']))

#Only sushi seems a bit high, remove sushi
df2 = df2.drop(['sushi'], axis=1)
df_loc_2 = df_loc_2.drop(['sushi'], axis=1)

0.5236268683293631
0.39230375616447377
0.3346637424528914
0.4882328490126407
0.409131943557723


In [37]:
#Write to csv
df2.to_csv('covid_restaurant_full_post_profiling.csv')
df_loc_2.to_csv('covid_restaurant_full_post_profiling_loc.csv')